<a href="https://colab.research.google.com/github/lytvyniuk/iowa_liquor_sales-Exploratory-Data-Analysis-/blob/stage/Test_task_iowa_liquor_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

**Initial Data Cleaning**

**Top 10 stores with the most bottles sold**

In [0]:
# select aggregated data on bottles_sold and vendors
%%bigquery --project protean-genius-271221 df
SELECT vendor_number, sum(bottles_sold)  as bottles_sold FROM `bigquery-public-data.iowa_liquor_sales.sales`  group by vendor_number order by bottles_sold desc limit 10

In [0]:
# select unique names of the stores , because some stores have same name code but slight difference in names (uppercase and lowercase etc), so it is needed to group correctly
%%bigquery --project protean-genius-271221 df_names
SELECT distinct vendor_number, vendor_name FROM `bigquery-public-data.iowa_liquor_sales.sales` 

In [0]:
# merging with names
df = pd.merge(df, df_names, on = "vendor_number", how='left').drop_duplicates(subset='vendor_number', keep="first")
df


In [0]:
sns.barplot(x = 'vendor_name', y = 'bottles_sold',  data = df, color="green")
plt.xticks(rotation=50, horizontalalignment='right')
plt.xlabel("Vendor name")
plt.ylabel("Bottles sold")
plt.title("Top 10 biggest vendors")
plt.show()

Let's see how many drinks are sold in different days, month.

In [0]:
%%bigquery --project protean-genius-271221 df_dates
SELECT bottles_sold, date FROM `bigquery-public-data.iowa_liquor_sales.sales` limit 10000

In [0]:
# date column to datetime
df_dates['date'] = pd.to_datetime(df_dates['date'], errors='coerce')
# select day of week, day of month, month from date
df_dates['Week_day'] = df_dates['date'].dt.weekday
df_dates['Month_day'] = df_dates['date'].dt.day
df_dates['Month'] = df_dates['date'].dt.month

# group by day of week
df_dates_agg = df_dates.groupby(['Week_day'])['bottles_sold'].sum().reset_index()
df_dates_agg = pd.DataFrame(df_dates_agg,columns=['Week_day', 'bottles_sold'])

In [0]:
plt2 = sns.barplot(x = 'Week_day', y = 'bottles_sold',  data = df_dates_agg, color="blue")
plt2.set_xticklabels(plt2.get_xticklabels(), rotation=50, horizontalalignment='right')
plt2.set(xlabel='Day of the week', ylabel='Bottles sold')

In [0]:
# group by day of month
df_dates_agg = df_dates.groupby('Month_day')['bottles_sold'].sum().reset_index()
df_dates_agg = pd.DataFrame(df_dates_agg,columns=['Month_day', 'bottles_sold'])
df_dates_agg

plt3 = sns.barplot(x = 'Month_day', y = 'bottles_sold',  data = df_dates_agg, color="blue")
plt3.set_xticklabels(plt3.get_xticklabels(), rotation=50, horizontalalignment='right')
plt3.set(xlabel='Day of the month', ylabel='Bottles sold')

In [0]:
# group by month
df_dates_agg = df_dates.groupby('Month')['bottles_sold'].sum().reset_index()
df_dates_agg = pd.DataFrame(df_dates_agg,columns=['Month', 'bottles_sold'])
df_dates_agg

plt4 = sns.barplot(x = 'Month', y = 'bottles_sold',  data = df_dates_agg, color="blue")
plt4.set_xticklabels(plt4.get_xticklabels(), rotation=50, horizontalalignment='right')
plt4.set(xlabel='Month', ylabel='Bottles sold')